# RSA Public Key Cryptosystem

The [RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) is a asymmetric cryptosystem, just like ElGamal. Was invented in 1977 by Rivest, Shamir and Adleman and relies on the difficulty of factorizing large prime numbers.

First Alice generates two large random primes *p* and *q* along with *N=pq*

In [1]:
from crypt import RandomPrime, xgcd, PrimesSieveEratosthenes, fastPowering

bits = 16

p = RandomPrime(bits, 40)
q = RandomPrime(bits, 40)
primes = list(PrimesSieveEratosthenes(1<<bits))

while p==q:
  q = RandomPrime(bits, 40)  
N = p*q
print("Two large primes:\np={}\nq={}\nN=pq={}".format(p, q, N))

Two large primes:
p=58313
q=44453
N=pq=2592187789


Then again Alice choose a number $e$ such that is coprime to the Carmichael function of N, this is $\lambda(N)$. This is the same as the least common multiple of $p-1$ and $q-1$, 

$$\lambda(N)=LCM(\lambda(p), \lambda(q))$$

but since $p$ and $q$ are prime numbers, the Carmichael totient function  of $p$ and $q$ are, respectively $p-1$ and $q-1$. Therefore 

$$\lambda(N)=LCM(p-1, q-1)$$

Therefore $e$ has to be in the range of 2 and $\lambda(N)$ and gcd between $e$ and $\lambda(N)$ is 1, i.e. they factorise with different numbers.

Let's begin by writing a function of the least common multiple

In [2]:
def LCM(a, b):
    g, _, _ = xgcd(a, b)
    return a*b//g

Now the full algorithm to generate $e$

In [3]:
from random import randrange

def RSA_choose_e(p, q):
    # p and q must be different primes

    lm = LCM(p-1, q-1)

    while True:
        e = randrange(2, lm)
        g, _, _ = xgcd(e, lm)
        if g==1:
            return e


In [4]:
e = RSA_choose_e(p, q)
print("e={}".format(e))

e=15397931


Alice sends *N* and *e* to Bob (the public key). Then bob can encrypt a message *m* and send it securely to Alice. 

In [5]:
m = 1234
c = fastPowering(m, e, N)

print("m={}\nciphertext of m is c={}".format(m, c))

m=1234
ciphertext of m is c=2116871445


Alice receives the ciphertext *c* and decrypts it.

In [6]:
from crypt import InverseMod

d = InverseMod(e, (p-1)*(q-1))
mp = fastPowering(c, d, N)

print("Recovered message m={}".format(mp))

Recovered message m=1234


## A better implementation of RSA

I've coded the same algorithm in a more compact way so key generation encryption and decryption is easier to implement in aplications.

In [7]:
from crypt import RSAKeyGenerator, RSAEncrypt, RSADecrypt

bits = 64
PublicKey, PrivateKey = RSAKeyGenerator(bits)

print(f"Public Key is (N, e)=({PublicKey[0]}, {PublicKey[1]})")
print(f"Private Key is (N, d)=({PrivateKey[0]}, {PrivateKey[1]})")

Public Key is (N, e)=(141437240279199282481413125071538008717, 7754970719310291013295796330747715243)
Private Key is (N, d)=(141437240279199282481413125071538008717, 9090511355638364784677135903623826947)


In [8]:
message = randrange(2, 1<<bits)
ciphertext = RSAEncrypt(message, PublicKey)
decrypted_message = RSADecrypt(ciphertext, PrivateKey)

print(f"A random message to send: {message}")
print(f"The encrypted ciphertext: {ciphertext}")
print(f"The decrypted ciphertext i.e. the message is {decrypted_message}")


A random message to send: 12621084038085508916
The encrypted ciphertext: 4628440639712269705731535002613312286
The decrypted ciphertext i.e. the message is 12621084038085508916


Try the above code with larger number of bits, e.g 128 or 256 to see that it works with larger prime numbers. The larger the prime number the better the securtity is